In [1]:
pip install -Uqq fastbook

Note: you may need to restart the kernel to use updated packages.


In case "Add persistent_workers class variable to FakeLoader " error

please downgrade py torch version by using -> `conda install pytorch==1.6.0 torchvision==0.7.0 -c pytorch` command

* https://github.com/fastai/fastai/pull/2878
* https://github.com/lgvaz/faststyle/pull/18


In [1]:
from sklearn.metrics import classification_report

from fastai.tabular.all import *
from fastai.basics import *
from fastai import *

import numpy as np
import pandas as pd
import glob
# torch.set_num_threads(10)
defaults.device = torch.device('cpu')

In [2]:
# load the baseline model
#learn.load('TubularLearner.fastAI._stage1')
deployed_path = "../Models/TubularLearner.retrain"
#deployed_path = "TubularLearner.fastAI._stage2.pth"
learner = load_learner(deployed_path)



In [3]:
learner.model

TabularModel(
  (embeds): ModuleList()
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(1900, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(1900, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=1900, out_features=200, bias=False)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=200, out_features=100, bias=False)
      (2): ReLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=100, out_features=2, bias=True)
    )
  )
)

In [29]:

df = pd.read_pickle("../datasets/truthset/AMPs_truthset.reps.plk")
ready_df =df[[ "reps"]]

## Chanage format

In [30]:
df_new = ready_df.reps.apply(pd.Series).astype(np.float64)
df_new.columns = df_new.columns.astype(str)
#to = TabularPandas(df_new , cont_names =reps_columns, y_block = CategoryBlock, splits=splits)

In [31]:
df_new

,0,1,2,3,4,5,6,7,8,9,...,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899
0,0.019923,-0.047088,0.007590,0.008612,-0.105413,0.032731,-0.297581,-0.010141,-0.018119,0.102418,...,0.047615,0.063532,-0.237826,0.039653,-0.031136,-0.006353,0.039390,0.007629,0.089423,0.022756
1,0.014724,-0.030932,0.007524,0.013040,0.047267,0.032513,-0.456108,0.001002,-0.018511,0.141110,...,0.040018,0.072904,-0.216747,0.028120,-0.036611,-0.004548,0.061547,0.045790,0.103151,0.000341
2,0.018701,-0.055086,-0.007485,0.018340,0.027769,0.043306,-0.380586,0.005666,-0.021887,0.109023,...,0.042081,0.087484,-0.214472,0.029637,-0.015207,-0.016494,0.060464,0.010968,0.061983,0.013685
3,0.019437,-0.031499,0.001900,0.012346,0.067398,0.023426,-0.267920,-0.002214,-0.018159,0.091275,...,0.051911,0.080598,-0.181252,0.030322,-0.044635,-0.007344,0.081058,0.017975,0.128263,0.028620
4,0.033317,-0.097852,0.010363,0.020239,-0.232632,0.049497,-0.299169,0.012850,-0.023980,0.337575,...,0.033375,0.120847,-0.100889,0.020196,-0.042631,0.023899,0.053246,0.001743,0.185660,0.020270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,0.016171,0.000104,0.001302,0.014256,0.017668,-0.001196,-0.343563,-0.265699,-0.016795,0.055140,...,0.024991,0.022828,-0.073042,0.017890,0.013226,-0.123560,0.154960,-0.013497,0.031413,-0.043297
58,0.015017,-0.036066,0.038192,0.003745,-0.003776,0.009892,-0.153868,-0.129352,-0.015966,0.075090,...,0.017711,0.006127,-0.039396,-0.001067,-0.054788,0.044370,0.132005,0.039209,-0.042818,-0.024418
59,0.032130,-0.000488,-0.014703,0.031483,0.013354,0.035091,-0.428796,0.013158,-0.035880,0.128336,...,0.044803,0.096227,-0.195344,0.034177,0.018314,-0.049155,0.140474,0.044257,0.071362,-0.001932
60,0.025432,0.006171,-0.012787,0.026035,-0.040039,0.050619,-0.472542,-0.001317,-0.029686,0.191039,...,0.035620,0.075275,-0.103784,0.032539,0.011790,0.012831,0.180955,0.092308,0.256334,-0.000305


## Predict
To get prediction on a new dataframe, you can use the test_d

In [32]:
dl = learner.dls.test_dl(df_new)
_preds,_none ,_y = learner.get_preds(dl=dl, with_decoded=True)

Basic Check

check these data exist in training set

# Predict on Shuffled sequences

In [10]:
df= pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
#df[df["length"] < 200] 

In [11]:
df

,ID,paper,seq,length,reps
0,ISGCock_Contig04_0915,PMC4864078,RQDRAHNQIDGQRPMGRLDKRKIAMKCTLSNQVFAKRRITYQTHKGQTPV,50,"[0.01679776795208454, -0.07345905154943466, 0.0018531358800828457, 0.01055986899882555, -0.06956648826599121, 0.02960575371980667, -0.29150453209877014, -0.011141696013510227, -0.0164033155888319, 0.08916237950325012, 0.08458603173494339, 0.010968605056405067, 0.04112812131643295, 0.007730279583483934, 0.014417233876883984, 0.00988777820020914, 0.035421740263700485, -0.007013504859060049, -0.1786070466041565, -0.020148323848843575, 0.003731424454599619, 0.041703954339027405, 0.0548442043364048, -0.12764130532741547, 0.020689815282821655, 0.05831281840801239, 0.08569350838661194, 0.02860361..."
1,ISGCock_Contig13_4610,PMC4864078,ICCLSTLMGNPRVKHLNKNNTYPKYIVLYLFHKLGFIQQLKHQC,44,"[0.015076708979904652, -0.050224486738443375, 0.00566816097125411, 0.013054276816546917, 0.12498209625482559, 0.026709938421845436, -0.31857830286026, -0.006246310193091631, -0.018232561647892, 0.09537849575281143, 0.12265881896018982, 0.014183806255459785, 0.05066995322704315, 0.09092161804437637, 0.016077512875199318, 0.01152981910854578, 0.015476514585316181, -0.03599875420331955, 0.10444874316453934, -0.015200425870716572, 0.008167939260601997, 0.03478241711854935, 0.06818510591983795, -0.036124568432569504, 0.02337823435664177, 0.032912254333496094, 0.07545920461416245, 0.021426292136..."
2,ISGCock_Contig16_2060,PMC4864078,VKHTKRTISYIYHKILRNSLMFTHINVHANGALK,34,"[0.020523976534605026, -0.03999472036957741, -0.008676423691213131, 0.018380343914031982, -0.03633490204811096, 0.0320831723511219, -0.3653090298175812, 0.002308246912434697, -0.02207029238343239, 0.09355314075946808, 0.23387080430984497, 0.017236748710274696, 0.040781646966934204, 0.10663942992687225, 0.01859077252447605, 0.014213756658136845, 0.028563503175973892, -0.05209049582481384, -0.03443988785147667, -0.01778305694460869, 0.012098332867026329, 0.037674784660339355, 0.05580299347639084, -0.10062112659215927, 0.03836312144994736, 0.11769463866949081, 0.0672989934682846, 0.0460695140..."
3,ISGCock_Contig16_4974,PMC4864078,KKIDKLIRPTVTLVHKSKFSESKAKILHTCQRFKKICPLLNRNW,44,"[0.02007628232240677, -0.05524342507123947, 0.01819263957440853, 0.012198265641927719, -0.018687721341848373, 0.027849018573760986, -0.21275494992733002, -0.00019348329806234688, -0.018473874777555466, 0.1106361597776413, 0.25107067823410034, 0.011788442730903625, 0.052943188697099686, 0.09616860747337341, 0.016118526458740234, 0.010019286535680294, 0.023470327258110046, -0.016887081786990166, 0.05189380794763565, -0.01595395617187023, 0.007148402277380228, 0.048173047602176666, 0.06571124494075775, -0.07286034524440765, 0.03498806059360504, 0.040984805673360825, -0.02916710264980793, 0.02..."
4,ISGCock_Contig07_3736,PMC4864078,SKMCINSRYSTYSCCMRFKKHVYSFNNGFCQ,31,"[0.022961627691984177, -0.03958016261458397, -0.0051064882427453995, 0.019761141389608383, -0.16499893367290497, 0.03426509350538254, -0.30065014958381653, 0.011957984417676926, -0.02363228239119053, 0.16380977630615234, 0.16033726930618286, 0.013369404710829258, 0.052276723086833954, 0.06975245475769043, 0.02058296836912632, 0.015751659870147705, 0.0293504036962986, -0.031605008989572525, -0.11130017787218094, -0.025151502341032028, 0.01198747381567955, 0.03925140202045441, 0.05175544321537018, -0.03918582946062088, 0.03501526638865471, -0.11543688178062439, 0.07330848276615143, 0.0382081..."
...,...,...,...,...,...
57,TR42258|c1_g1_i1,hal-02965337,KSSVSLFFFPIVFFARLFVTALLRYRLFGLLMCSTLSPRWAFWV,44,"[0.014877736568450928, 0.00022438698215410113, -0.002712303539738059, 0.013208281248807907, 0.033270493149757385, 0.020772254094481468, -0.3263610005378723, -0.07785046100616455, -0.017812687903642654, 0.10032114386558533, -0.1033627837896347, 0.011208795011043549, 0.03223970904946327, 0.09667403250932693, 0.01945313811302185, 0.010947594419121742, 0.02236204221844673, -0.012472672387957573, 0.08771192282438278, -0.01426319032907486, 0.0106107909232378, 0.

In [14]:
def predict(_df ,condition):
    #df = _df.loc[_df.length > 30 ].copy() 
    #df = _df.loc[_df.length <= 30 ].copy()
    if condition == 0:
        df = _df.copy()
    elif condition == -1 :
        df = _df.loc[_df.length <= 30 ].copy()
    else:
        df = _df.loc[_df.length > condition ].copy() # select records with lenght > 
    #print(df.columns)
    ready_df = df[[ "reps"]]
    df_new = ready_df.reps.apply(pd.Series).astype(np.float64)
    df_new.columns = df_new.columns.astype(str)
    dl = learner.dls.test_dl(df_new)
    _preds,_none ,_y = learner.get_preds(dl=dl, with_decoded=True)
    df['class'] = _y
    #df.drop(columns=['reps'],inplace =True)
    accuracy = (len(df[df["class"] == 1])/len(df) )*100
    #print("predict AMP :"+str(len(df[df["class"] == 0])))
    #print("predict NonAMP:"+str(len(df[df["class"] == 1])))
    #print("predict NonAMP (in percentage):"+str(accuracy))
    return len(df[df["class"] == 1])

## Retrain with <= 200 AA

In [ ]:
result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
# shuff1
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
result_df.loc['shuff1', 'no_con']=predict(df_,0)
result_df.loc['shuff1', '>200']= predict(df_,200)
result_df.loc['shuff1', '>100']=predict(df_,100)
result_df.loc['shuff1', '>70']=predict(df_,70)
result_df.loc['shuff1', '>50']=predict(df_,50)
result_df.loc['shuff1', '>30']=predict(df_,30)
result_df.loc['shuff1', '30<=']=predict(df_,-1)
# shuff2
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_43.reps.plk")
result_df.loc['shuff2', 'no_con']=predict(df_,0)
result_df.loc['shuff2', '>200']= predict(df_,200)
result_df.loc['shuff2', '>100']=predict(df_,100)
result_df.loc['shuff2', '>70']=predict(df_,70)
result_df.loc['shuff2', '>50']=predict(df_,50)
result_df.loc['shuff2', '>30']=predict(df_,30)
result_df.loc['shuff2', '30<=']=predict(df_,-1)
# shuff3
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_44.reps.plk")
result_df.loc['shuff3', 'no_con']=predict(df_,0)
result_df.loc['shuff3', '>200']= predict(df_,200)
result_df.loc['shuff3', '>100']=predict(df_,100)
result_df.loc['shuff3', '>70']=predict(df_,70)
result_df.loc['shuff3', '>50']=predict(df_,50)
result_df.loc['shuff3', '>30']=predict(df_,30)
result_df.loc['shuff3', '30<=']=predict(df_,-1)
# shuff4
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_45.reps.plk")
index = "shuff4"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff5
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_46.reps.plk")
index = "shuff5"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff6
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_47.reps.plk")
index = "shuff6"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff7
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_48.reps.plk")
index = "shuff7"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff8
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_49.reps.plk")
index = "shuff8"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff9
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_50.reps.plk")
index = "shuff9"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff10
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_51.reps.plk")
index = "shuff10"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)

result_df

In [6]:
result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,24,4,7,15,16,21,3
shuff2,25,4,7,11,18,23,2
shuff3,23,4,6,12,14,20,3
shuff4,18,4,6,9,11,15,3
shuff5,23,4,7,13,13,21,2
shuff6,22,4,7,13,13,18,4
shuff7,16,4,6,9,9,12,4
shuff8,25,4,7,12,15,22,3
shuff9,29,4,7,11,21,26,3
shuff10,19,4,6,11,12,17,2


## Retrain with <= 30 AA

In [7]:
deployed_path = "../Models/TubularLearner._30.retrain"
#deployed_path = "TubularLearner.fastAI._stage2.pth"
learner = load_learner(deployed_path)

In [ ]:
result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
# shuff1
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
result_df.loc['shuff1', 'no_con']=predict(df_,0)
result_df.loc['shuff1', '>200']= predict(df_,200)
result_df.loc['shuff1', '>100']=predict(df_,100)
result_df.loc['shuff1', '>70']=predict(df_,70)
result_df.loc['shuff1', '>50']=predict(df_,50)
result_df.loc['shuff1', '>30']=predict(df_,30)
result_df.loc['shuff1', '30<=']=predict(df_,-1)
# shuff2
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_43.reps.plk")
result_df.loc['shuff2', 'no_con']=predict(df_,0)
result_df.loc['shuff2', '>200']= predict(df_,200)
result_df.loc['shuff2', '>100']=predict(df_,100)
result_df.loc['shuff2', '>70']=predict(df_,70)
result_df.loc['shuff2', '>50']=predict(df_,50)
result_df.loc['shuff2', '>30']=predict(df_,30)
result_df.loc['shuff2', '30<=']=predict(df_,-1)
# shuff3
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_44.reps.plk")
result_df.loc['shuff3', 'no_con']=predict(df_,0)
result_df.loc['shuff3', '>200']= predict(df_,200)
result_df.loc['shuff3', '>100']=predict(df_,100)
result_df.loc['shuff3', '>70']=predict(df_,70)
result_df.loc['shuff3', '>50']=predict(df_,50)
result_df.loc['shuff3', '>30']=predict(df_,30)
result_df.loc['shuff3', '30<=']=predict(df_,-1)
# shuff4
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_45.reps.plk")
index = "shuff4"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff5
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_46.reps.plk")
index = "shuff5"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff6
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_47.reps.plk")
index = "shuff6"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff7
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_48.reps.plk")
index = "shuff7"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff8
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_49.reps.plk")
index = "shuff8"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff9
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_50.reps.plk")
index = "shuff9"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff10
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_51.reps.plk")
index = "shuff10"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)


In [9]:
result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,18,3,6,10,11,15,3
shuff2,19,3,6,12,12,16,3
shuff3,20,2,5,12,14,20,0
shuff4,10,2,5,5,5,8,2
shuff5,18,4,7,10,12,17,1
shuff6,22,4,7,12,13,19,3
shuff7,17,3,6,9,9,14,3
shuff8,12,2,5,7,7,11,1
shuff9,21,3,6,6,15,20,1
shuff10,17,2,5,11,11,16,1


## Retrain with whoel dataset but 1900 layers

In [19]:
deployed_path = "../Models/TubularLearner.1900_200.retrain"
#deployed_path = "TubularLearner.fastAI._stage2.pth"
learner = load_learner(deployed_path)

result_df = pd.DataFrame( columns=["no_con",">200",">100",">70",">50",">30","30<="],
    index = ['shuff1','shuff2', 'shuff3',"shuff4","shuff5","shuff6","shuff7","shuff8","shuff9","shuff10"])
# shuff1
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_42.reps.plk")
result_df.loc['shuff1', 'no_con']=predict(df_,0)
result_df.loc['shuff1', '>200']= predict(df_,200)
result_df.loc['shuff1', '>100']=predict(df_,100)
result_df.loc['shuff1', '>70']=predict(df_,70)
result_df.loc['shuff1', '>50']=predict(df_,50)
result_df.loc['shuff1', '>30']=predict(df_,30)
result_df.loc['shuff1', '30<=']=predict(df_,-1)
# shuff2
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_43.reps.plk")
result_df.loc['shuff2', 'no_con']=predict(df_,0)
result_df.loc['shuff2', '>200']= predict(df_,200)
result_df.loc['shuff2', '>100']=predict(df_,100)
result_df.loc['shuff2', '>70']=predict(df_,70)
result_df.loc['shuff2', '>50']=predict(df_,50)
result_df.loc['shuff2', '>30']=predict(df_,30)
result_df.loc['shuff2', '30<=']=predict(df_,-1)
# shuff3
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_44.reps.plk")
result_df.loc['shuff3', 'no_con']=predict(df_,0)
result_df.loc['shuff3', '>200']= predict(df_,200)
result_df.loc['shuff3', '>100']=predict(df_,100)
result_df.loc['shuff3', '>70']=predict(df_,70)
result_df.loc['shuff3', '>50']=predict(df_,50)
result_df.loc['shuff3', '>30']=predict(df_,30)
result_df.loc['shuff3', '30<=']=predict(df_,-1)
# shuff4
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_45.reps.plk")
index = "shuff4"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff5
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_46.reps.plk")
index = "shuff5"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff6
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_47.reps.plk")
index = "shuff6"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff7
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_48.reps.plk")
index = "shuff7"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff8
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_49.reps.plk")
index = "shuff8"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff9
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_50.reps.plk")
index = "shuff9"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)
# shuff10
df_ = pd.read_pickle("../datasets/truthset/AMPs_truthset_51.reps.plk")
index = "shuff10"
result_df.loc[index, 'no_con']=predict(df_,0)
result_df.loc[index, '>200']= predict(df_,200)
result_df.loc[index, '>100']=predict(df_,100)
result_df.loc[index, '>70']=predict(df_,70)
result_df.loc[index, '>50']=predict(df_,50)
result_df.loc[index, '>30']=predict(df_,30)
result_df.loc[index, '30<=']=predict(df_,-1)


In [20]:
result_df

,no_con,>200,>100,>70,>50,>30,30<=
shuff1,22,4,7,14,15,19,3
shuff2,26,4,7,12,23,26,0
shuff3,20,4,6,12,14,19,1
shuff4,13,4,6,8,8,12,1
shuff5,23,4,7,13,13,21,2
shuff6,24,4,7,13,16,22,2
shuff7,16,4,6,10,10,14,2
shuff8,21,4,7,12,15,20,1
shuff9,18,4,4,6,12,16,2
shuff10,18,4,7,11,12,17,1
